# 0.1 WavesExecuter
## Basic Usage

This `QurriumInstance` does not any post-processing feature but provides a workdlow that can execute multiple at once.


### a. Import the instances


In [1]:
from qurry import WavesExecuter

experiment_workflow = WavesExecuter()

/home/nccu/Documents/.homejupyter/qurryplayground/qurrium_docs/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### b. Preparing quantum circuit

Remember for any quantum circuit, you need to make sure that they all contain classical registers.
Otherwise, `WavesExecuter` will raise an `ValueError` when you try to execute the circuit.


In [2]:
from qiskit import QuantumCircuit
from qurry.recipe import TrivialParamagnet, GHZ


def make_neel_circuit(n):
    qc = QuantumCircuit(n)
    for i in range(0, n, 2):
        qc.x(i)
    return qc


circuits_dict = {}

for i in range(2, 13, 2):
    circuits_dict[f"trivial_paramagnet_{i}_ms"] = TrivialParamagnet(i)
    circuits_dict[f"ghz_{i}_ms"] = GHZ(i)
    circuits_dict[f"neel_{i}_ms"] = make_neel_circuit(i)

for name, circuit in circuits_dict.items():
    circuit.measure_all()

In [3]:
no_creq = TrivialParamagnet(2)
print(no_creq.draw())

try:
    experiment_workflow.measure([no_creq], shots=1024)
except ValueError as e:
    print(f"Error: {e}")

     ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     └───┘
Error: | No classical register in ALL circuits, counts will be empty. Please add classical register to the circuit. (Don't be frustrated, I did the same thing on unit test. It made me confused and thought what's wrong for a while before ('_').)


### c. Execute the circuit


#### i. Directly input the circuit

After executing, it will return a uuid of experiment. You can use this uuid to get the result of the experiment.


In [4]:
exp1 = experiment_workflow.measure(list(circuits_dict.values()), shots=1024)
exp1

'1ffe4f74-1d81-4729-85b3-ad0cea13d86d'

Each experiment result will be stored in a container `.exps`.


In [5]:
experiment_workflow.exps[exp1]

<WavesExecuterExperiment(exp_id=1ffe4f74-1d81-4729-85b3-ad0cea13d86d, 
  WavesExecuterArguments(exp_name='experiment.waves_executer'),
  Commonparams(exp_id='1ffe4f74-1d81-4729-85b3-ad0cea13d86d', target_keys=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18], shots=1024, backend=<BasicSimulator('basic_simulator')>, run_args={}, transpile_args={}, tags=(), default_analysis=[], save_location=PosixPath('.'), filename='', files={}, serial=None, summoner_id=None, summoner_name=None, datetimes=DatetimeDict({'build': '2025-05-22 14:44:31', 'run.001': '2025-05-22 14:44:31'})),
  unused_args_num=0,
  analysis_num=0))>

`WavesExecuter` does not provide any post-processing feature.
So `.analyze()` **will not provide any useful information**, but

**The Answer to the Ultimate Question of Life, The Universe, and Everything**


In [6]:
report01 = experiment_workflow.exps[exp1].analyze()
report01

| ultimate_question: 


<WavesQurryAnalysis(
  serial=0,
  AnalysisInput(ultimate_question=''),
  AnalysisContent(ultimate_answer=42, dummy=-100)),
  unused_args_num=0
  )>

In [7]:
main01, side_product01 = report01.export()
main01

{'ultimate_answer': 42,
 'input': {'ultimate_question': ''},
 'header': {'serial': 0,
  'datetime': '2025-05-22 14:44:31',
  'summoner': None,
  'log': {}}}

#### ii. Add the circuits to container `.waves`, then call them later.

Since we have executed an experiment, the circuit we input in `exp1` is stored in the container `.waves` with serial number `0`.


In [8]:
experiment_workflow.waves

WaveContainer({
  0: <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x73f949ba47d0>,
  1: <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x73f9c80ea030>,
  2: <qurry.recipe.simple.cat.GHZ object at 0x73f949cbfdd0>,
  3: <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x73f9c8110dd0>,
  4: <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x73f949ba5ac0>,
  5: <qurry.recipe.simple.cat.GHZ object at 0x73f94b9dce00>,
  6: <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x73f949bcee70>,
  7: <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x73f94b955370>,
  8: <qurry.recipe.simple.cat.GHZ object at 0x73f949aa8170>,
  9: <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x73f949aabc80>,
  10: <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x73f949ae0170>,
  11: <qurry.recipe.simple.cat.GHZ object at 0x73f949ae01a0>,
  12: <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x73f949ae0200>,
  13: <qurry.re

But we can also add the circuit to the container `.waves` with a custom name.
The name should be unique, otherwise it will be overwritten.
The method `add` will return the actual name of the circuit in the container.


In [9]:
for k, v in circuits_dict.items():
    print(experiment_workflow.add(v, k))

trivial_paramagnet_2_ms
ghz_2_ms
neel_2_ms
trivial_paramagnet_4_ms
ghz_4_ms
neel_4_ms
trivial_paramagnet_6_ms
ghz_6_ms
neel_6_ms
trivial_paramagnet_8_ms
ghz_8_ms
neel_8_ms
trivial_paramagnet_10_ms
ghz_10_ms
neel_10_ms
trivial_paramagnet_12_ms
ghz_12_ms
neel_12_ms


If there is a circuit with the same name, it will be replaced by the new one.


In [10]:
print(experiment_workflow.add(circuits_dict["trivial_paramagnet_2_ms"], "ghz_8_ms"))
print(experiment_workflow.waves["ghz_8_ms"])

ghz_8_ms
        ┌───┐ ░ ┌─┐   
   q_0: ┤ H ├─░─┤M├───
        ├───┤ ░ └╥┘┌─┐
   q_1: ┤ H ├─░──╫─┤M├
        └───┘ ░  ║ └╥┘
meas: 2/═════════╩══╩═
                 0  1 


Otherwise, you will need to use `replace="duplicate"` to prevent it from being replaced.


In [11]:
duplicated_case01 = experiment_workflow.add(
    circuits_dict["trivial_paramagnet_2_ms"], "ghz_8_ms", replace="duplicate"
)
print(duplicated_case01)
print(experiment_workflow.waves[duplicated_case01])

ghz_8_ms.37
        ┌───┐ ░ ┌─┐   
   q_0: ┤ H ├─░─┤M├───
        ├───┤ ░ └╥┘┌─┐
   q_1: ┤ H ├─░──╫─┤M├
        └───┘ ░  ║ └╥┘
meas: 2/═════════╩══╩═
                 0  1 


Now we have prepared the circuit and stored it in the container `.waves`.


In [12]:
experiment_workflow.waves

WaveContainer({
  0: <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x73f949ba47d0>,
  1: <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x73f9c80ea030>,
  2: <qurry.recipe.simple.cat.GHZ object at 0x73f949cbfdd0>,
  3: <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x73f9c8110dd0>,
  4: <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x73f949ba5ac0>,
  5: <qurry.recipe.simple.cat.GHZ object at 0x73f94b9dce00>,
  6: <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x73f949bcee70>,
  7: <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x73f94b955370>,
  8: <qurry.recipe.simple.cat.GHZ object at 0x73f949aa8170>,
  9: <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x73f949aabc80>,
  10: <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x73f949ae0170>,
  11: <qurry.recipe.simple.cat.GHZ object at 0x73f949ae01a0>,
  12: <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x73f949ae0200>,
  13: <qurry.re

Finally, we can execute the circuit and get the result.


In [13]:
exp2 = experiment_workflow.measure(list(circuits_dict.keys()), shots=1024)
exp2

'a062fee6-6f23-406d-8498-a54f99d79d60'

In [14]:
experiment_workflow.exps[exp2]

<WavesExecuterExperiment(exp_id=a062fee6-6f23-406d-8498-a54f99d79d60, 
  WavesExecuterArguments(exp_name='experiment.waves_executer'),
  Commonparams(exp_id='a062fee6-6f23-406d-8498-a54f99d79d60', target_keys=['trivial_paramagnet_2_ms', 'ghz_2_ms', 'neel_2_ms', 'trivial_paramagnet_4_ms', 'ghz_4_ms', 'neel_4_ms', 'trivial_paramagnet_6_ms', 'ghz_6_ms', 'neel_6_ms', 'trivial_paramagnet_8_ms', 'ghz_8_ms', 'neel_8_ms', 'trivial_paramagnet_10_ms', 'ghz_10_ms', 'neel_10_ms', 'trivial_paramagnet_12_ms', 'ghz_12_ms', 'neel_12_ms'], shots=1024, backend=<BasicSimulator('basic_simulator')>, run_args={}, transpile_args={}, tags=(), default_analysis=[], save_location=PosixPath('.'), filename='', files={}, serial=None, summoner_id=None, summoner_name=None, datetimes=DatetimeDict({'build': '2025-05-22 14:44:38', 'run.001': '2025-05-22 14:44:38'})),
  unused_args_num=0,
  analysis_num=0))>

In [15]:
report02 = experiment_workflow.exps[exp2].analyze(
    "What is the answer to the ultimate question of life, the universe, and everything"
)
report02

| ultimate_question: What is the answer to the ultimate question of life, the universe, and everything


<WavesQurryAnalysis(
  serial=0,
  AnalysisInput(ultimate_question='What is the answer to the ultimate question of life, the universe, and everything'),
  AnalysisContent(ultimate_answer=42, dummy=-100)),
  unused_args_num=0
  )>

### d. take `counts`

In [ ]:
experiment_workflow.exps[exp2].afterwards.counts[:3]

[{'00': 243, '10': 270, '01': 250, '11': 261},
 {'00': 501, '11': 523},
 {'01': 1024},
 {'1101': 71,
  '1000': 57,
  '0010': 58,
  '0000': 70,
  '0001': 70,
  '0101': 62,
  '0100': 68,
  '0011': 66,
  '1010': 63,
  '1001': 74,
  '1100': 50,
  '0111': 66,
  '0110': 68,
  '1110': 74,
  '1111': 55,
  '1011': 52},
 {'1111': 494, '0000': 530},
 {'0101': 1024},
 {'010101': 18,
  '110000': 23,
  '001011': 16,
  '010000': 11,
  '010110': 14,
  '000011': 15,
  '001101': 17,
  '100100': 20,
  '110110': 16,
  '100011': 19,
  '111100': 12,
  '010111': 19,
  '101111': 16,
  '011101': 16,
  '110101': 16,
  '011111': 14,
  '101000': 20,
  '011001': 12,
  '010001': 19,
  '100110': 24,
  '110010': 13,
  '001111': 24,
  '011010': 14,
  '110001': 14,
  '011011': 19,
  '010011': 12,
  '100001': 17,
  '101010': 16,
  '010010': 11,
  '111101': 20,
  '011100': 16,
  '111111': 18,
  '101110': 17,
  '111010': 20,
  '000010': 17,
  '100000': 17,
  '000100': 19,
  '011000': 20,
  '101011': 20,
  '001000': 15,
  

### e. Export them after all


In [17]:
exp1_id, exp1_files_info = experiment_workflow.exps[exp1].write(
    save_location=".",  # where to save files
)
exp1_files_info

{'folder': 'experiment.waves_executer.003',
 'qurryinfo': 'experiment.waves_executer.003/qurryinfo.json',
 'args': 'experiment.waves_executer.003/args/experiment.waves_executer.003.id=1ffe4f74-1d81-4729-85b3-ad0cea13d86d.args.json',
 'advent': 'experiment.waves_executer.003/advent/experiment.waves_executer.003.id=1ffe4f74-1d81-4729-85b3-ad0cea13d86d.advent.json',
 'legacy': 'experiment.waves_executer.003/legacy/experiment.waves_executer.003.id=1ffe4f74-1d81-4729-85b3-ad0cea13d86d.legacy.json',
 'reports': 'experiment.waves_executer.003/reports/experiment.waves_executer.003.id=1ffe4f74-1d81-4729-85b3-ad0cea13d86d.reports.json',
 'reports.tales.dummy': 'experiment.waves_executer.003/tales/experiment.waves_executer.003.id=1ffe4f74-1d81-4729-85b3-ad0cea13d86d.dummy.reports.json'}

## Post-Process Availablities and Version Info


In [18]:
from qurry.process import AVAIBILITY_STATESHEET

AVAIBILITY_STATESHEET

 | Qurry version: 0.12.2.dev2
---------------------------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability ................... Python Cython Rust   JAX   
 - randomized_measure
   - entangled_entropy.entropy_core_2 ....... Yes    Depr.  Yes    No    
   - entangle_entropy.purity_cell_2 ......... Yes    Depr.  Yes    No    
   - entangled_entropy_v1.entropy_core ...... Yes    Depr.  Yes    No    
   - entangle_entropy_v1.purity_cell ........ Yes    Depr.  Yes    No    
   - wavefunction_overlap.echo_core_2 ....... Yes    Depr.  Yes    No    
   - wavefunction_overlap.echo_cell_2 ....... Yes    Depr.  Yes    No    
   - wavefunction_overlap_v1.echo_core ...... Yes    Depr.  Yes    No    
   - wavefunction_overlap_v1.echo_cell ...... Yes    Depr.  Yes    No    
 - hadamard_test
   - purity_echo_core ....................... Yes    No     Yes    No    
 - magnet_square
   - magnsq_core ............................ Yes    No     No     No  